In [1]:
import pandas as pd
import pyodbc

In [2]:
def sqlserver_sql(q, server = 'SQL2017', db = 'IPL_IRASV6_STAGE'):
    driver = '{SQL Server Native Client 11.0}'
    conn = pyodbc.connect("Driver="+driver+";Server="+server+";Database="+db+";Trusted_Connection=yes;")

    # performing query to database
    df = pd.read_sql_query(q,conn)

    conn.close()
    return df

In [3]:
server2 = 'SQL2012'
db2 = 'IPL_INTERCHANGE_PROD'

Linear Queries

In [6]:
table_stg = 'IRASCustomData1'
table_inter = 'GeohazardPOF'

q_stg = f"""select t.*, ll.LineName from {table_stg} t
            join StationSeries ss on t.beginstationSeriesid = ss.id
            join lineloop ll on ss.lineloopid = ll.id
    """

vt_q_stg = f"""select t.*, vts.BeginStationSeriesId, vts.BeginStationNum, vts.EndStationNum, ll.LineName from VirtualTableDatalistdomain t
                join VirtualTableStationing vts on t.VirtualTableStationingId = vts.VirtualTableStationingId and t.VirtualTableId = vts.VirtualTableId
                join VirtualTable vt on t.VirtualTableId = vt.Id
                join StationSeries ss on vts.BeginStationSeriesId = ss.Id
                join LineLoop ll on ss.LineLoopId = ll.Id
                where vt.virtualtablename = '{table_stg}'
            """
q_inter = f"""select t.*, ll.LineName from ipl_{table_inter} t
            join ipl_stationseries ss on t.beginstationseriesid = ss.id
            join ipl_lineloop ll on ss.lineloopid = ll.id
    """
el_inter = f"""select t.*, ll.LineName from ipl_{table_inter} t
            join ipl_stationseries ss on t.stationseriesid = ss.id
            join ipl_lineloop ll on ss.lineloopid = ll.id
    """

stg_data = sqlserver_sql(q_stg)
inter_data = sqlserver_sql(q_inter, server=server2, db=db2)

display(stg_data.count().iloc[0])

display(inter_data.count().iloc[0])

241

241

In [8]:
# stg_data.loc[:,'SourceId'] = stg_data.Id.astype(object)
temp = inter_data.merge(stg_data,left_on='Id',right_on='SourceId', suffixes=('_inter','_stg'))
# temp.loc[:,'STN_DIFF'] = temp.loc[:,['BeginStationNum_inter','BeginStationNum_stg']].apply(lambda x: x.BeginStationNum_inter.round(0) == x.BeginStationNum_stg.round(0), axis=1)
temp.loc[:,'PCT_DIFF'] = temp.loc[:,['BeginStationNum_inter','BeginStationNum_stg']].apply(lambda x: (abs(x.BeginStationNum_inter-x.BeginStationNum_stg)*100.0)/x.BeginStationNum_inter, axis=1)
# temp.loc[:,'PCT_DIFF'] = temp.loc[:,['StationNum','BeginStationNum']].apply(lambda x: (abs(x.StationNum-x.BeginStationNum)*100.0)/x.BeginStationNum, axis=1)

# temp.loc[lambda x: x.PCT_DIFF> 1,['Id_inter','Id_stg','SourceId','BeginStationSeriesId_inter','BeginStationSeriesId_stg','PCT_DIFF','STN_DIFF','BeginStationNum_inter','BeginStationNum_stg']].round(0)
# temp.loc[lambda x: x.PCT_DIFF> 1,['Id','VirtualTableStationingId','SourceId','BeginStationSeriesId_inter','BeginStationSeriesId_stg','PCT_DIFF','STN_DIFF','BeginStationNum_inter','BeginStationNum_stg']].round(0)
temp.loc[lambda x: x.PCT_DIFF> 1,['Id_inter','Id_stg','SourceId','StationSeriesId','BeginStationSeriesId','PCT_DIFF','StationNum','BeginStationNum']].round(0)

C:\Users\armando_borjas\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,Id_inter,Id_stg,SourceId,StationSeriesId,BeginStationSeriesId,PCT_DIFF,StationNum,BeginStationNum


In [10]:
temp.filter(regex='.*Station.*')
temp.loc[:,['BeginStationNum_inter','BeginStationNum_stg']]

,BeginStationNum_inter,BeginStationNum_stg
0,565.535,565.53502
1,15263.614,15263.61449
2,13824.352,13824.35244
3,13525.505,13525.50543
4,15615.258,15615.25850
...,...,...
236,8055.839,8055.83926
237,29016.227,29016.22793
238,28962.296,28962.29692
239,1690.198,1690.19805


Point Queries

In [211]:
table_stg = 'inlineinspectionjoint'
table_inter = 'inlineinspectionjoint'

q_stg = f"""select t.*, ll.LineName from {table_stg} t
            join StationSeries ss on t.StationSeriesId = ss.id
            join lineloop ll on ss.lineloopid = ll.id
    """

q_inter = f"""select t.*, ll.LineName from ipl_{table_inter} t
            join ipl_stationseries ss on t.StationSeriesId = ss.id
            join ipl_lineloop ll on ss.lineloopid = ll.id
    """

stg_data = sqlserver_sql(q_stg)
inter_data = sqlserver_sql(q_inter, server=server2, db=db2)

display(stg_data.count().iloc[0])

display(inter_data.count().iloc[0])

0

0

In [210]:
temp = inter_data.merge(stg_data,left_on='Id',right_on='SourceId', suffixes=('_inter','_stg'))
temp.loc[:,'STN_DIFF'] = temp.loc[:,['StationNum_inter','StationNum_stg']].apply(lambda x: x.StationNum_inter.round(0) == x.StationNum_stg.round(0), axis=1)
temp.loc[:,'PCT_DIFF'] = temp.loc[:,['StationNum_inter','StationNum_stg']].apply(lambda x: (abs(x.StationNum_inter-x.StationNum_stg)*100.0)/x.StationNum_inter, axis=1)

temp.loc[lambda x: x.PCT_DIFF> 1,['Id_inter','Id_stg','SourceId','StationSeriesId_inter','StationSeriesId_stg','PCT_DIFF','STN_DIFF','StationNum_inter','StationNum_stg']].round(0)

C:\Python36\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


,Id_inter,Id_stg,SourceId,StationSeriesId_inter,StationSeriesId_stg,PCT_DIFF,STN_DIFF,StationNum_inter,StationNum_stg


In [208]:
temp.head(100).loc[lambda x: x.PCT_DIFF > 0,['Id_inter','Id_stg','SourceId','StationSeriesId_inter','StationSeriesId_stg','PCT_DIFF','STN_DIFF','StationNum_inter','StationNum_stg']].round(0)

,Id_inter,Id_stg,SourceId,StationSeriesId_inter,StationSeriesId_stg,PCT_DIFF,STN_DIFF,StationNum_inter,StationNum_stg
0,1,1.0,1,439,439.0,0.0,True,888.0,888.0
1,2,2.0,2,223,223.0,0.0,True,10701.0,10701.0
2,3,3.0,3,306,306.0,0.0,True,27596.0,27596.0
4,5,5.0,5,30,30.0,0.0,True,6292.0,6292.0
5,6,6.0,6,52,52.0,0.0,True,278.0,278.0
...,...,...,...,...,...,...,...,...,...
94,222,222.0,222,153,153.0,0.0,True,4531.0,4531.0
96,224,224.0,224,330,330.0,0.0,True,22624.0,22624.0
97,225,225.0,225,330,330.0,0.0,True,36042.0,36042.0
98,226,226.0,226,330,330.0,0.0,True,47588.0,47588.0
